### An attempt to integrate PredPreyGrassEnv into ray rllib 2.9.3

In [1]:
#NUMBER III
#define policies in dict with PolicySpec
#add width and height parameters to the environment

from environments.predpreygrass_env import PredPreyGrassEnv
from config.config_rllib import configuration

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

check_env(PredPreyGrassEnv(configuration))

def env_creator(configuration):
    return PredPreyGrassEnv(configuration)  # return an env instance

register_env("pred_prey_grass", env_creator)

policy1 = PolicySpec()
policy2 = PolicySpec()

policies = { 
    "policy1": policy1,
    "policy2": policy2
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id.startswith("predator_"):
        return "policy1"
    if agent_id.startswith("prey_"):
        return "policy2"
    else:
        raise ValueError(f"Unexpected agent ID: {agent_id}")

config = (
    PPOConfig()
    .environment(env="pred_prey_grass")
    .framework("torch")
    .rollouts(
        create_env_on_local_worker=True,
        batch_mode="truncate_episodes", #"complete_episodes", 
        num_rollout_workers=0,
        rollout_fragment_length= "auto",
    )
    .debugging(seed=0,log_level="ERROR")
    .training(model={
        "fcnet_hiddens" : [64, 64], 
        "_disable_preprocessor_api": True,
        "conv_filters": [[32, [8, 8], 4], [64, [4, 4], 2], [512, [1, 1], 1]] # Copilot
        }
    )
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
    #.build()
)


algo = config.build()

env = PredPreyGrassEnv(configuration=configuration)
obs, _ = env.reset()

#print(obs)
"""
action0 = algo.compute_single_action(obs["predator_0"], policy_id="policy1")
action1 = algo.compute_single_action(obs["predator_1"], policy_id="policy1")
action2 = algo.compute_single_action(obs["predator_2"], policy_id="policy1")
action3 = algo.compute_single_action(obs["prey_3"], policy_id="policy2")
action4 = algo.compute_single_action(obs["prey_4"], policy_id="policy2")
action5 = algo.compute_single_action(obs["prey_5"], policy_id="policy2")
print("action0",action0)
print("action1",action1)
print("action2",action2)
print("action3",action3)
print("action4",action4)
print("action5",action5)
"""
algo.train()

algo.stop()



pygame 2.3.0 (SDL 2.24.2, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
/home/doesburg/Dropbox/03_marl_code/PredPreyGrass/.conda/lib/python3.11/site-packages/ray/rllib/evaluation/env_runner_v2.py
env_runner_v2, 277 * self._active_episodes:  {}
env_runner_v2, 287 * self._large_batch_threshold:  40000
rollout_ops.py, 81 * worker_set <ray.rllib.evaluation.worker_set.WorkerSet object at 0x7f65e93c0fd0>
rollout_ops.py, 81 * worker_set.local_worker() <ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f65ebb5ba90>
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_runner_v2, 710 * self._multiple_episodes_in_batch:  True
env_ru

ValueError: ('Batches sent to postprocessing must only contain steps from a single trajectory.', SampleBatch(54: ['obs', 'new_obs', 'actions', 'rewards', 'terminateds', 'truncateds', 'infos', 'eps_id', 'unroll_id', 'agent_index', 't', 'vf_preds', 'action_dist_inputs', 'action_logp']))